<a href="https://colab.research.google.com/github/AanLetna7025/NLP2Query/blob/main/Nlp2Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install peft

In [ ]:
import pandas as pd
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM


# DataSet

In [ ]:
schema = {
    "users": ["id", "name", "email", "signup_date"],
    "orders": ["order_id", "user_id", "order_date", "amount"],
    "products": ["product_id", "name", "price", "stock", "category_id"],
    "categories": ["category_id", "category_name"],
    "reviews": ["review_id", "user_id", "product_id", "rating", "review_text", "review_date"],
    "payments": ["payment_id", "order_id", "payment_date", "payment_method", "payment_status"]
}


In [ ]:
#  Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/MyDrive/Colab Notebooks/nl2sql_dataset (1).csv'


Mounted at /content/drive


In [ ]:
df = pd.read_csv(path)

df.head()

,input,output
0,Get all users who signed up today,SELECT * FROM users WHERE signup_date = CURREN...
1,Find products with price greater than 100,SELECT * FROM products WHERE price > 100;
2,Count number of orders placed by each user,"SELECT user_id, COUNT(*) FROM orders GROUP BY ..."
3,Get the names of all users,SELECT name FROM users;
4,List all products that are out of stock,SELECT * FROM products WHERE stock = 0;


In [ ]:
from datasets import Dataset
dataset=Dataset.from_pandas(df)

def format_prompt(example):
    return {
        "text": f"Human: {example['input']}\nAssistant: {example['output']}"
    }

formatted_dataset = dataset.map(format_prompt)

Map:   0%|          | 0/637 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-0.5B",
    device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B", use_fast=False)


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
SCHEMA = """Tables:
- users(id, name, email, signup_date)
- orders(order_id, user_id, order_date, amount)
- products(product_id, name, price, stock, category_id)
- categories(category_id, category_name)
- reviews(review_id, user_id, product_id, rating, review_text, review_date)
- payments(payment_id, order_id, payment_date, payment_method, payment_status)
"""

def tokenize(example):
    tokenized_input= tokenizer(example["text"],
                                 padding="max_length",
                                 truncation=True,
                                 max_length=512)
    tokenized_input["labels"] = tokenized_input["input_ids"].copy()
    return tokenized_input

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/637 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./qwen-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    warmup_steps=100,
    weight_decay=0.01,
    report_to="none",
    fp16=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
#import os
#os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer.train()

Step,Training Loss
10,19.976300
20,13.173600
30,0.351800
40,0.096700
50,0.066500
60,0.050900
70,0.057800
80,0.061000
90,0.055900
100,0.052400


TrainOutput(global_step=957, training_loss=0.386553538365673, metrics={'train_runtime': 1643.279, 'train_samples_per_second': 1.163, 'train_steps_per_second': 0.582, 'total_flos': 1810521381666816.0, 'train_loss': 0.386553538365673, 'epoch': 3.0})

# session


In [ ]:
save_path = "/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/chat_template.jinja',
 '/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/vocab.json',
 '/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/merges.txt',
 '/content/drive/MyDrive/Colab Notebook/Models/qwen-finetuned1/added_tokens.json')

In [ ]:
prompt = "<|user|>\n" + SCHEMA + "\n\nQuery: List all products in 'Electronics' \n<|assistant|>\n"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<|user|>
Tables:
- users(id, name, email, signup_date)
- orders(order_id, user_id, order_date, amount)
- products(product_id, name, price, stock, category_id)
- categories(category_id, category_name)
- reviews(review_id, user_id, product_id, rating, review_text, review_date)
- payments(payment_id, order_id, payment_date, payment_method, payment_status)


Query: List all products in 'Electronics' 
<|assistant|>
Assistant: SELECT * FROM products WHERE category_id = (SELECT id FROM categories WHERE name = 'Electronics');
